In [1]:
import sklearn
import sklearn.preprocessing

In [2]:
import pandas as pd

snitz_ravia = pd.read_csv("Data/Challenge/Snitz_Ravia_Definition.csv").dropna()
snitz_ravia["Mixture Label"] = snitz_ravia["Mixture Label"].astype(int)
snitz_ravia

,Dataset,Mixture Label,CID,CID.1,CID.2,CID.3,CID.4,CID.5,CID.6,CID.7,...,CID.48,CID.49,CID.50,CID.51,CID.52,CID.53,CID.54,CID.55,CID.56,CID.57
1,Snitz 1,1,6501.0,264.0,2879.0,7685.0,7731.0,326.0,7888.0,61138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Snitz 1,2,240.0,93009.0,323.0,8148.0,7762.0,3314.0,460.0,6184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Snitz 1,3,7710.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Snitz 1,4,31276.0,93009.0,11002.0,323.0,7966.0,8148.0,7632.0,22201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Snitz 1,5,10890.0,93009.0,11002.0,6982.0,323.0,8797.0,7966.0,8148.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Ravia,116,7799.0,31244.0,1031.0,7363.0,14286.0,8025.0,19310.0,7793.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,Ravia,117,6590.0,1031.0,176.0,180.0,7363.0,7150.0,14286.0,8025.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,Ravia,118,1068.0,1031.0,180.0,3776.0,14286.0,61138.0,8025.0,6561.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,Ravia,119,7799.0,3776.0,264.0,62336.0,6569.0,10448.0,261.0,5284503.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
def extract_mixture_labels_and_cids(df):
    result = dict()
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        cids = [int(cid) for cid in row[cid_columns].tolist() if cid != 0]
        assert len(set(cids)) == len(cids)
        
        
        result[mixture_label] = cids

    return result

def extract_mixture_labels(df):
    result = []
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        result.append(mixture_label)

    return result
        
snitz_ravia_mixtures = extract_mixture_labels_and_cids(snitz_ravia)
len(snitz_ravia_mixtures), next(iter(snitz_ravia_mixtures.items()))

(183,
 ('Snitz 1_1', [6501, 264, 2879, 7685, 7731, 326, 7888, 61138, 8030, 1183]))

In [21]:
snitz_ravia_experiments = pd.read_csv("Data/Challenge/TrainingData_mixturedist.csv")
snitz_ravia_experiments = snitz_ravia_experiments[snitz_ravia_experiments['Dataset'].notnull()]
snitz_ravia_experiments = snitz_ravia_experiments[snitz_ravia_experiments['Dataset'] != 'Bushdid']
snitz_ravia_experiments

,Dataset,Mixture 1,Mixture 2,Experimental Values
0,Snitz 1,1.0,2.0,0.604167
1,Snitz 1,1.0,3.0,0.651042
2,Snitz 1,1.0,5.0,0.505208
3,Snitz 1,1.0,6.0,0.411458
4,Snitz 1,1.0,7.0,0.562500
...,...,...,...,...
261,Ravia,86.0,96.0,0.406200
262,Ravia,87.0,97.0,0.390600
263,Ravia,88.0,98.0,0.421900
264,Ravia,89.0,99.0,0.421900


In [24]:
snitz_ravia_experiments["Label 1"] = snitz_ravia_experiments.apply(lambda x: x["Dataset"]+"_"+str(int(x["Mixture 1"])),axis=1)
snitz_ravia_experiments["Label 2"] = snitz_ravia_experiments.apply(lambda x: x["Dataset"]+"_"+str(int(x["Mixture 2"])),axis=1)
snitz_ravia_experiments["CIDS 1"] = snitz_ravia_experiments.apply(lambda x: snitz_ravia_mixtures[x["Label 1"]],axis=1)
snitz_ravia_experiments["CIDS 2"] = snitz_ravia_experiments.apply(lambda x: snitz_ravia_mixtures[x["Label 2"]],axis=1)
snitz_ravia_experiments

,Dataset,Mixture 1,Mixture 2,Experimental Values,Label 1,Label 2,CIDS 1,CIDS 2
0,Snitz 1,1.0,2.0,0.604167,Snitz 1_1,Snitz 1_2,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[240, 93009, 323, 8148, 7762, 3314, 460, 6184,..."
1,Snitz 1,1.0,3.0,0.651042,Snitz 1_1,Snitz 1_3,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...",[7710]
2,Snitz 1,1.0,5.0,0.505208,Snitz 1_1,Snitz 1_5,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[10890, 93009, 11002, 6982, 323, 8797, 7966, 8..."
3,Snitz 1,1.0,6.0,0.411458,Snitz 1_1,Snitz 1_6,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[7710, 31276, 10890, 240, 93009, 11002, 6982, ..."
4,Snitz 1,1.0,7.0,0.562500,Snitz 1_1,Snitz 1_7,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[93009, 460, 443158, 6544]"
...,...,...,...,...,...,...,...,...
261,Ravia,86.0,96.0,0.406200,Ravia_86,Ravia_96,"[1068, 6590, 31244, 1031, 3776, 7363, 7150, 80...","[1068, 6590, 31244, 180, 3776, 7363, 7150, 611..."
262,Ravia,87.0,97.0,0.390600,Ravia_87,Ravia_97,"[1068, 6590, 31244, 176, 3776, 7150, 8025, 193...","[6590, 31244, 1031, 7150, 19310, 6561, 7793, 2..."
263,Ravia,88.0,98.0,0.421900,Ravia_88,Ravia_98,"[1068, 3776, 7363, 61138, 8025, 19310, 22386, ...","[6590, 180, 3776, 7363, 8025, 19310, 12020, 22..."
264,Ravia,89.0,99.0,0.421900,Ravia_89,Ravia_99,"[1031, 180, 7363, 7150, 8025, 6561, 443158, 26...","[1068, 31244, 180, 7363, 7150, 8025, 7793, 443..."


In [4]:
train_df = pd.concat([snitz_ravia,bushdid]).fillna(0)
train_df

,Dataset,Mixture Label,CID,CID.1,CID.2,CID.3,CID.4,CID.5,CID.6,CID.7,...,CID.48,CID.49,CID.50,CID.51,CID.52,CID.53,CID.54,CID.55,CID.56,CID.57
1,Snitz 1,1,6501.0,264.0,2879.0,7685.0,7731.0,326.0,7888.0,61138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Snitz 1,2,240.0,93009.0,323.0,8148.0,7762.0,3314.0,460.0,6184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Snitz 1,3,7710.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Snitz 1,4,31276.0,93009.0,11002.0,323.0,7966.0,8148.0,7632.0,22201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Snitz 1,5,10890.0,93009.0,11002.0,6982.0,323.0,8797.0,7966.0,8148.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,Bushdid,261,16666.0,19310.0,62465.0,7848.0,440917.0,7991.0,1060.0,8051.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516,Bushdid,262,6561.0,7921.0,7654.0,24915.0,261.0,7888.0,999.0,8030.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517,Bushdid,262,1127.0,10821.0,8858.0,8148.0,3776.0,8048.0,7731.0,31249.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
518,Bushdid,264,2969.0,264.0,8048.0,6054.0,4133.0,8635.0,10560.0,12348.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def extract_mixture_labels_and_cids(df):
    result = dict()
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        cids = [int(cid) for cid in row[cid_columns].tolist() if cid != 0]
        assert len(set(cids)) == len(cids)
        
        
        result[mixture_label] = cids

    return result

def extract_mixture_labels(df):
    result = []
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        result.append(mixture_label)

    return result
        
train_results = extract_mixture_labels_and_cids(train_df)
len(train_results), next(iter(train_results.items()))

(443,
 ('Snitz 1_1', [6501, 264, 2879, 7685, 7731, 326, 7888, 61138, 8030, 1183]))

In [6]:
test_df = pd.read_csv("Data/Challenge/Mixure_Definitions_test_set.csv")
test_results = extract_mixture_labels_and_cids(test_df)
len(test_results), next(iter(test_results.items()))

(92, ('1', [10890, 62465, 12180, 7799, 6054, 12367, 61016, 7410, 7749, 10722]))

In [7]:
leaderboard_df = pd.read_csv("Data/Challenge/Mixure_Definitions_Leaderboard_set.csv")
leaderboard_results = extract_mixture_labels(leaderboard_df)
len(leaderboard_results), leaderboard_results[0]

(87, 'Snitz 1_1')

Combine the correct data from "TrainingData_mixturedist.csv" with 

In [19]:
# Load the Excel file
file_path = 'Data/bushdid-tables2.xlsx'
data = pd.read_excel(file_path, sheet_name=' ')

# Extract the subject columns
subject_columns = [col for col in data.columns if col.startswith('subject')]

# Convert the 'right'/'wrong' responses to binary (1 for 'right', 0 for 'wrong')
for col in subject_columns:
    data[col] = data[col].map({'right': 1, 'wrong': 0})

# Calculate the mean discriminability for each row
data['mean_discriminability'] = data[subject_columns].mean(axis=1)

# Group by Test UID and calculate the mean discriminability for each mixture
mixture_discriminability = data.groupby('Test UID')['mean_discriminability'].mean()
mixture_discriminability

Test UID
1                               0.576923
2                               0.692308
3                               0.269231
4                               0.538462
5                               0.576923
                                  ...   
261                             0.730769
262                             0.538462
263                             0.961538
264                             0.807692
Control Discrimination Tests         NaN
Name: mean_discriminability, Length: 265, dtype: float64

Combine this and the Snitz Ravia to get the actually correct training data.

In [10]:
train_cids = set()
for label, cids in train_results.items():
    train_cids.update(cids)
    
len(train_cids)

162

In [ ]:
test_cids
for label, cids in test.items():
    train_cids.update(cids)